# Homework: Decipherment

## Our main efforts to improve the baseline:
- Implement find_ext_order function to find better extension order. 
- Rewrite the score function which is used in beam_search function. 3-4 x faster. 
- Multiprocessing. 10 x faster with 64 core VM compared to single core implementation.
- Customize the ext_limits

## Contents  <a id='part-0'>
-------------------
- [Part 1: Load the libraries](#part-1)
- [Part 2: Code from the default notebook](#part-2)
- [Part 3: Implementation for Reference 3 to find the optimal extension order](#part-3)
- [Part 4: Baseline with better extension order and faster score function](#part-4)
- [Part 5: Test case](#part-5)
- [Part 6: Multi-processing](#part-6)
- [Part 7: Decipher Zodiac Killer cipher](#part-7)

## Part 1. Load the libraries <a id='part-1'></a>

In [2]:
%load_ext autoreload
%autoreload 2
from collections import defaultdict, Counter
import collections
import pprint
import math
import bz2
from ngram import *
import sys, string, os
import copy
import pickle
#from joblib import Parallel, delayed
import itertools
from multiprocessing import Process,Pool, cpu_count
import datetime, time, random
pp = pprint.PrettyPrinter(width=45, compact=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


###  [Back to contents](#part-0)

## Part 2. Code from the default notebook <a id='part-2'></a>

First let us read in the cipher text from the `data` directory:

In [2]:
def read_file(filename):
    if filename[-4:] == ".bz2":
        with bz2.open(filename, 'rt', encoding='utf-8-sig') as f:
            content = f.read()
            f.close()
    else:
        with open(filename, 'r', encoding='utf-8-sig') as f:
            content = f.read()
            f.close()
    return content

cipher = read_file("data/cipher.txt")
# print(cipher)

While using `get_statistics`, make sure that `cipher=True` is set when the input is a ciphertext.

In [3]:
def get_statistics(content, cipher=True):
    stats = {}
    content = list(content)
    split_content = [x for x in content if x != '\n' and x!=' ']
    length = len(split_content)
    symbols = set(split_content)
    uniq_sym = len(list(symbols))
    freq = collections.Counter(split_content)
    rel_freq = {}
    for sym, frequency in freq.items():
        rel_freq[sym] = (frequency/length)*100
        
    if cipher:
        stats = {'content':split_content, 'length':length, 'vocab':list(symbols), 'vocab_length':uniq_sym, 'frequencies':freq, 'relative_freq':rel_freq}
    else:
        stats = {'length':length, 'vocab':list(symbols), 'vocab_length':uniq_sym, 'frequencies':freq, 'relative_freq':rel_freq}
    return stats

In [4]:
cipher_desc = get_statistics(cipher, cipher=True)
plaintxt = read_file("data/default.wiki.txt.bz2")
plaintxt_desc = get_statistics(plaintxt, cipher=False)

In [5]:
%%time
sequence = 'In a few cases, a multilingual artifact has been necessary to facilitate decipherment, the Rosetta Stone being the classic example. Statistical techniques provide another pathway to decipherment, as does the analysis of modern languages derived from ancient languages in which undeciphered texts are written. Archaeological and historical information is helpful in verifying hypothesized decipherments.'

# lm = LM("data/6-gram-wiki-char.lm.bz2", n=6, verbose=True)
lm = LM("data/6-gram-wiki-char.lm.bz2", n=6, verbose=False)

Reading language model from data/6-gram-wiki-char.lm.bz2...


CPU times: user 24.5 s, sys: 971 ms, total: 25.5 s
Wall time: 25.5 s


Done.


###  [Back to contents](#part-0)

## Part 3. Implementation for Reference 3 to find the optimal extension order <a id='part-3'></a>

This is the implementation of the reference 'Improved Decipherment of Homophonic Ciphers'. The goal is to find the best extension order. As the paper mentioned, it is important to find a better optimal extension order which is helpful to improve both the speed and accuracy. If the correct answer has been pruned out at a very early stage, it is not quite possible that the text can be fully deciphered since the following score is computed based on a wrong partial deciphered text. With beam size of 10000, we experimented with several sets of extension orders. With the help of ground truth, we are able to monitor when the correct answer has been pruned. If we follow the frequency order of those symbols, the correct answer can be pruned as early as the fourth or fifth iteration. Finally, I chose the weights \[1,1,1,1,2,3\] suggested by Anoop in a discussion post.

In [1]:
# define a function to calculate the sharp_n for the specified n-gram order
def find_sharp_n(cipher_desc, symbols_found, n_order):
    '''
    finds the #n for order n_order
    cipher_desc -- cipher statistics
    symbols_found -- list of single character string,
                     specifies the list of symbols have been placed in the extention order
    n_order -- int, specifies the order of n-gram
    '''
    sharp_n = 0
    for i in range(len(cipher_desc['content'])-n_order+1):
        for j in range(i, i+n_order, 1):
            if cipher_desc['content'][j] not in symbols_found:
                break
            if j == (i+n_order-1):
                sharp_n += 1
                #print(cipher_desc['content'][i:i+n_order])
    return sharp_n            

Use a beam search to find the optimal extension order. The code below is pretty similar to the beam_search function. Most of the code is copied from it. For simplicity, the variable name might not quite make sense.

In [2]:
def find_ext_order(cipher_desc, topn=100, weights=[1,1,1,1,2,3]):
    '''
    finds the best order of deciphering cipher symbols (find best extention order)
    cipher_desc -- cipher statistics
    topn -- int, number of best trees we want to keep during iteration, aka beam size
    weights -- list of int, weight for #n, n varies from 1 to 6 in the case of 6 gram
    '''
    # symbols already found with score
    Hs = [([], 0)]
    # hypothesis extended symbols with score
    Ht = []
    # initialize the cardinality (number of unique cipher text)
    cardinality = 0
    # if no weight is specified for unigram, use the most frequent symbol as the starting point
    if weights[0] == 0:
        cardinality += 1
        Hs.append(([sorted(cipher_desc['frequencies'], key=cipher_desc['frequencies'].get, reverse=True)[0]], 0))
    # list of cipher symbols
    Ve = sorted(cipher_desc['frequencies'], key=cipher_desc['frequencies'].get, reverse=True)
    while cardinality < cipher_desc['vocab_length']:
        for phi, previous_score in Hs:
            for e in Ve:
                phi_prime = copy.deepcopy(phi)
                if e in phi_prime:
                    continue
                else:
                    phi_prime.append(e)
                    this_score = 0
                    for i in range(len(weights)):
                        this_score += weights[i]*find_sharp_n(cipher_desc, phi_prime, i+1)
                    Ht.append((phi_prime, this_score))
        # prune the histogram
        Ht = sorted(Ht, key=lambda x:x[1], reverse=True)[:topn]                    
        cardinality += 1
        Hs = copy.deepcopy(Ht)
        Ht.clear()
    return sorted(Hs, key=lambda x:x[1], reverse=True)

In [8]:
# this may takes a while (3.5 min on my machine)
ext_orders = find_ext_order(cipher_desc, topn=100, weights=[1,1,1,1,2,3])
ext_order = ext_orders[0][0]

###  [Back to contents](#part-0)

## Part 4. Baseline with better extension order and faster score function <a id='part-4'></a>

The first change I made to the baseline was to rewrite the score function to optimize the running speed. The change I made is to score the newly fixed symbol plantext character pair and corresponding influenced previously fixed plantext character based on the previous score instead of scoring the whole bitstring in each iteration. For instance, 'oooo...o' -> 'oooo..xo'. The new score can be calculated by adding unigram score of 'x' to the previous score, substracting unigram score of 'o' following 'x' and bigram score of '<\s>' from the previous score, and adding bigram score of 'o' following 'x' and trigram score of '<\s>' to the previous score. With this approach, the running time was improved to 20 minitues from more than one hour with a beamsize of 10000 on my machine with i7 7700k cpu. And the computed score is almost same as the score computed with score_bit_string function (the difference is within 0.00000000001). Another approach to speed up the whole process we tried is multiprocessing. We will talk about that in details in the following notebook.

In [12]:
def score(cipher, phi, new_f, new_e, previous_score):
    '''
    scores the phi_prime based on the previous score, returns a float
    cipher -- list of single character string
    phi -- dictionary, old mapping e->[f]
    new_f -- single-character string, extended symbol
    previous_score -- float, old score for phi
    '''
    mapping = phi
    new_score = previous_score
    # for the first iteration, the previous score should be -2.545382 instead of 0
    # this is because the score of an empty string is not 0 whiling scoring with bitstring
    # the value can be obtained by calling lm.score_bitstring('thisisatest', '...........')
    if len(phi)==0:
        new_score += -2.545382
    lm_state = lm.begin()
    old_lm_state = lm.begin()
    # this Flag is used to track if a newly-fixed character affects the previously-fixed character
    triggerChangeFlag = 0
    for i in range(len(cipher)):
        char = cipher[i]
        # if this is a previously fixed character and not influenced by the newly-fixed character
        # we only need to track the lm_state and old lm_state, no need to compute the score
        if (char in mapping.keys()) and (triggerChangeFlag==0):
            token = mapping[char]
            ngram = lm_state + (token,)
            while len(ngram)> 0:
                if ngram in lm.table:
                    lm_state = ngram[-lm.history:]
                    break
                else: #backoff
                    ngram = ngram[1:]
            if len(ngram)==0:
                lm_state = ()
            old_lm_state = lm_state
        # if this is a previously fixed character and influenced by the newly-fixed character
        # substract the old score and add the new score to the previous score.
        elif (char in mapping.keys()) and (triggerChangeFlag>0):
            token = mapping[char]
            old_lm_state, old_logprob = lm.score(old_lm_state, token)
            new_score -= old_logprob
            lm_state, logprob = lm.score(lm_state, token)
            new_score += logprob
            triggerChangeFlag -= 1
        # if this is a newly-fixed charater, simply add the new score to the previous score
        elif char == new_f:
            (lm_state, logprob) = lm.score(lm_state, new_e)
            new_score += logprob
            triggerChangeFlag = 5
            old_lm_state = ()
        # if this is a unknown character, there is no influence on the previous score
        else:
            lm_state = ()
            old_lm_state = ()
            triggerChangeFlag = 0
    # treat the end tag '<\s>' as previously fixed character
    if triggerChangeFlag:
        new_score -= lm.end(old_lm_state)
        new_score += lm.end(lm_state)
    return new_score

The beam_search we implemented is based on the pseudo code mentioned in the assignment. The main change I made was to use customized ext_limit for each plaintext character instead of a general value. The details can be found in the following notebook.

In [15]:
def beam_search(cipher, ext_order, ext_limits, topn=1):
    '''
    finds the mappings between cipher char and plaintext char, returns the mapping dictionary
    ext_order -- list, the unigram char list sorted by their count DESC
    ext_limits -- int, defines maximum number of cipher char can be mapped to a plaintext char
    topn -- int, defines the number of dictionaries we want to keep while pruning
    '''
    print('Number of unique symbols in cipher:', len(ext_order))
    # mapping relationships already found with score
    Hs = [(defaultdict(dict), 0)]
    # hypothesis mapping relationships with score
    Ht = []
    # initialize the cardinality (number of unique cipher text)
    cardinality = 0
    # list of plaintext characters
    Ve = [chr(i) for i in range(97, 123, 1)]
    while cardinality < len(ext_order):
        f = ext_order[cardinality]
        print('Working on symbol: ', f, '(%s)' % (cardinality+1))
        for phi, previous_score in Hs:
            for e in Ve:
                phi_prime = copy.deepcopy(phi)
                new_map = {f: e}
                phi_prime.update(new_map)
                counts = len([v for k, v in phi_prime.items() if v == e])
                ext_limit = ext_limits[e]
                if counts <= ext_limit:
                    Ht.append((phi_prime, score(cipher, phi, f, e, previous_score)))
        # prune the histogram
        Ht = sorted(Ht, key=lambda x:x[1], reverse=True)[:topn]
        cardinality += 1
        Hs = copy.deepcopy(Ht)
        Ht.clear()
        #print('Current score: ', Hs[0][1], 'Worst score: ', Hs[min(len(Hs)-1, topn-1)][1])
        #print(Hs)
    return sorted(Hs, key=lambda x:x[1], reverse=True)

###  [Back to contents](#part-0)

## Part 5. Test case <a id='part-5'></a>

Before deciphering the Zodiac Killer cipher, test the algorithm with a simple one to one mapping test case <br>
Plaintext: `defendtheeastwallofthecastle` <br>
Cipher: `giuifgceiiprctpnnduceiqprcni` <br>

In [16]:
one_to_one_cipher = 'giuifgceiiprctpnnduceiqprcni'
one_to_one_cipher_desc = get_statistics(one_to_one_cipher, cipher=True)
one_to_one_ext_order = find_ext_order(one_to_one_cipher_desc)[0][0]
one_to_one_ext_limits = dict()
for e in [chr(i) for i in range(97, 123, 1)]:
    one_to_one_ext_limits[e] = 1
one_to_one_mappings = beam_search(one_to_one_cipher_desc['content'], one_to_one_ext_order,\
                                  one_to_one_ext_limits, 500)
one_to_one_mapping = one_to_one_mappings[0][0]
one_to_one_decipher_text = ''
for char in one_to_one_cipher_desc['content']:
    one_to_one_decipher_text += one_to_one_mapping[char]
print('Deciphered result: ', one_to_one_decipher_text)

Number of unique symbols in cipher: 12
Working on symbol:  i (1)
Working on symbol:  p (2)
Working on symbol:  r (3)
Working on symbol:  c (4)
Working on symbol:  t (5)
Working on symbol:  n (6)
Working on symbol:  e (7)
Working on symbol:  q (8)
Working on symbol:  u (9)
Working on symbol:  d (10)
Working on symbol:  g (11)
Working on symbol:  f (12)
Deciphered result:  defendtheeastwallofthecastle


We didn't come up with a homophobic cipher. The reason is that our algorithm is able to decipher the Zodiac with a beamsize of 10000 after fixing three ground true mapping relationships (result not shown). The symbol error rate for that is ~5%. With a beamsize of 10000, the algorithm cannot correctly decipher the text all by its own. But it is enough to prove the baseline algorithm works.

###  [Back to contents](#part-0)

## Part 6. Multi-processing <a id='part-6'></a>

To furthur improve the running speed, we tried multiprocessing. <br>
**NOTE: This multi-processing version only works for linux or MAC operating system. It doesn't work for Windows system. Please run beam_search function instead of beam_search_mp if you are using a Windows machine.** <br>
This multi-processing implementation makes experimenting with beam size of 1M possible. With a 64 core linux instance, we are able to finish the whole process with 6h 30min. Compared to the running time mentioned in the original 2013 Nuhn paper, our implementation is faster than theirs (if we ignore the difference in the CPUs). They were using 128 core machine and the whole process takes more than 5 hours. <br>
Note that this multi-processing implementation is also memory-hungry. With 2M beamsize, the RAM must be larger than 256G for 128 core machine.

In [20]:
def parallel_fn(Ve, phi, f, cipher, previous_score, ext_limits):
    ret = []
    for e in Ve:
        phi_prime = copy.deepcopy(phi)
        new_map = {f: e}
        phi_prime.update(new_map)
        counts = len([v for k, v in phi_prime.items() if v == e])
        if counts <= ext_limits[e]:
            ret.append((phi_prime, score(cipher, phi, f, e, previous_score)))
    return ret

def beam_search_mp(cipher, ext_order, ext_limits, topn=1):

    # initialization
    Hs = [(defaultdict(dict), 0)]
    Ht = []
    cardinality = 0
    Ve = [chr(i) for i in range(97, 123, 1)]
    
    while cardinality < len(ext_order):
        f = ext_order[cardinality]
        print('Working on symbol: ', f, '(%s)' % (cardinality+1))
        
        mainStart = time.time()
        result = []
        p = Pool(cpu_count())
             
        for phi, previous_score in Hs: 
            result.append(p.apply_async(parallel_fn, args=(Ve, phi, f, cipher, previous_score, ext_limits))) 
                            
        p.close() 
        p.join()  

        Ht = []
        for subp in result:
            Ht += subp.get()
            
        if cardinality < 10 or cardinality >= 40:
            topn = 100000
        else:
            topn = 1000000
    
        # prune the histogram
        mainEnd = time.time()
        print ('Running Time for this symbol: %0.2f seconds.' % (mainEnd-mainStart))
        Ht = sorted(Ht, key=lambda x:x[1], reverse=True)[:topn]    
        
        cardinality += 1
        Hs = copy.deepcopy(Ht)
        Ht.clear()        
    return sorted(Hs, key=lambda x:x[1], reverse=True)

###  [Back to contents](#part-0)

## Part 7. Decipher Zodiac Killer cipher <a id='part-7'></a>

As briefly mentioned above, we use different ext_limit for different plaintext symbol. We use the following `ext_limits` to limit the ext_order. The goal is to customize the `ext_limit` for each plaintext character so that more symbols can be mapped to the more frequent plaintext character. The `ext_limit` is calculated by multiplying the relative frequency of the paintext character in wiki text by the number of unique cipher symbols. We believe this can dramatically improve the running time since the required amount of computation in each iteration is minimized. We use the ceiling instead of floor to ensure the `ext_limit` is large enough. We also add one to each ext_limit so that we can still be able to get close to the correct answer. For instance, the limit for 'e' is 7 in the case of Zodiac Killer task. If the correct answer was pruned and 'e' has been used up, the algorithm will make it worse. if the correct answer was pruned. Of course this approach might not be perfect if less frequent plaintext character is mapped to more cipher symbols. But even that is the case, the error rate should not increase significantly since the frequency of those less frequent plaintext should not be large (if the character distribution of the plaintext in the cipher task is similar to that of the one used to train the language model).

In [21]:
ext_limits = dict()
for e in [chr(i) for i in range(97, 123, 1)]:
    ext_limits[e] = math.ceil(plaintxt_desc['relative_freq'][e]*cipher_desc['vocab_length']/100)+1
print(ext_limits)

{'g': 2, 'c': 2, 'q': 1, 'o': 4, 'b': 1, 'a': 5, 'v': 1, 'm': 2, 's': 4, 'u': 2, 'e': 7, 'f': 2, 'k': 1, 'r': 4, 'x': 1, 'h': 3, 'j': 1, 't': 5, 'i': 4, 'l': 3, 'n': 4, 'y': 1, 'd': 3, 'w': 1, 'z': 1, 'p': 2}


Thanks to our improvement on the score function and the multiprocessing implementation, we are able to try larger beam size. We made several attempts. Here is the result. <br>
**64 vCPU VM on AZURE** Beam size: 1M; Running time: 6h 30min; Symbol error rate: 0.694; The correct leaf was pruned at iteration 13. <br>
**96 vCPU VM on AWS** Beam size: 1M for the first 10 iterations, 2M for the next 30 iterations, 1M afterwards; Running time: 5h 55min; Symbol error rate: 0.546; The correct leaf was pruned at iteration 15. <br>
We believe that if we have enough time and computation resources we would be able to find the final answer. But for now, the best we can achieve is 0.546. <br>
With SER of 0.546, the text starts to make sense. After the first run, we didn't freeze some mappings which might look correct. If we do so and make a another run, I am pretty sure we can find the closer decipher based on the fact the we can achieve SER of 0.05 after freezing three mappings. The reason we didn't do that is because that is some sort of cheating. Although the sentences start to make sense, we still cannot make sure that is the correct mapping without knowing the ground truth.

In [ ]:
# constant_ext_limit = 4
# ext_limits = {}
# for e in [chr(i) for i in range(97, 123, 1)]:
#     ext_limits[e] = constant_ext_limit

In [26]:
%%time
# single core version
# mappings = beam_search(cipher_desc['content'], ext_order, ext_limits, 1000000)
# multiprocessing version
mappings = beam_search_mp(cipher_desc['content'], ext_order, ext_limits, 1000000)

Working on symbol:  E (1)
Running Time for this symbol: 2.53 seconds.
Best score:  -10.7856218 Worst score:  -29.749483999999995
gold score -10.785621800000001
Working on symbol:  ∑ (2)
Running Time for this symbol: 2.65 seconds.
Best score:  -20.517739599999995 Worst score:  -59.14061499999999
gold score -24.181161000000003
Working on symbol:  B (3)
Running Time for this symbol: 2.63 seconds.
Best score:  -32.11996419999999 Worst score:  -100.08261399999998
gold score -38.09201220000001
Working on symbol:  P (4)
Running Time for this symbol: 5.16 seconds.
Best score:  -40.82754000000006 Worst score:  -133.53752555999998
gold score -48.728844000000024
Working on symbol:  º (5)
Running Time for this symbol: 63.11 seconds.
Best score:  -54.011017099999975 Worst score:  -81.53140200000003
gold score -62.340047100000035
Working on symbol:  ∫ (6)
Running Time for this symbol: 161.73 seconds.
Best score:  -66.89316288000005 Worst score:  -85.72715639999998
gold score -74.86408880000002
Worki

In [27]:
mapping = mappings[0][0]
decipher_text = ''
for char in cipher_desc['content']:
    decipher_text += mapping[char]
print(decipher_text)

ilikekillsneheahleraceneaitisthrnconnaissuroopdntthinkillinewilderresacoenosmeetrecansaretitsharaattineposnernarelodillcokillsaretoiaeesretresherhatcosilliteaqhampnceitseerenrestaochraeatsitedonsmocksandwitoaeiolchpretthesshnstiacorpwhenitieswillremarooniahisrdicauttellsopshirekilledwillrecarerdelarasiwillnoteiradonrdairerecentedonwillcodthelaitawtomuchhrdcollactineadeleresnoordansaslideareasiesaretohhiss
